In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf

In [3]:
engine = create_engine('sqlite:///TEST_DB.db')

In [4]:
df = yf.download('AAPL', start= '2022-01-01', end ='2022-11-01')

[*********************100%***********************]  1 of 1 completed


In [5]:
df

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2022-01-03,179.076599,179.932588,174.845913,174.963974,104487900
2022-01-04,176.803818,179.991605,176.233164,179.686603,99310400
2022-01-05,172.100876,177.266264,171.825390,176.715291,94537600
2022-01-06,169.227951,172.474769,168.873752,169.916666,96904000
2022-01-07,169.395172,171.333423,168.273546,170.103569,86709100
...,...,...,...,...,...
2022-10-25,150.504990,150.653192,147.560889,148.282092,74732300
2022-10-26,147.551041,150.159240,146.256807,149.141648,88194300


In [6]:
df.to_sql('AAPL', engine)

209

In [10]:
max_date = pd.read_sql('SELECT MAX(Date) FROM AAPL', engine).values[0][0]

In [11]:
max_date

In [13]:
df[df.index > max_date].to_sql('AAPL', engine, if_exists='append')

TypeError: Invalid comparison between dtype=datetime64[ns] and NoneType

In [14]:
pd.read_sql('AAPL', engine)

,Date,"('Close', 'AAPL')","('High', 'AAPL')","('Low', 'AAPL')","('Open', 'AAPL')","('Volume', 'AAPL')"
0,NaT,179.076599,179.932588,174.845913,174.963974,104487900
1,NaT,176.803818,179.991605,176.233164,179.686603,99310400
2,NaT,172.100876,177.266264,171.825390,176.715291,94537600
3,NaT,169.227951,172.474769,168.873752,169.916666,96904000
4,NaT,169.395172,171.333423,168.273546,170.103569,86709100
...,...,...,...,...,...,...
204,NaT,150.504990,150.653192,147.560889,148.282092,74732300
205,NaT,147.551041,150.159240,146.256807,149.141648,88194300
206,NaT,143.055817,147.254623,142.393889,146.286432,109180200
207,NaT,153.864044,155.602839,146.039446,146.414859,164762400


In [15]:
def sql_importer(data, table):
    max_date = pd.read_sql(f'SELECT MAX(Date) FROM {table}', engine).values[0][0]
    print(max_date)
    new_rows = data[data.index > max_date]
    new_rows.to_sql(table, engine, if_exists='append')
    print(str(len())) + 'new rows imported to DB'

In [16]:
sql_importer(df, 'AAPL')

None


TypeError: Invalid comparison between dtype=datetime64[ns] and NoneType

In [ ]:
def sql_importer(symbol):
    max_date = pd.read_sql(f'SELECT MAX(Date) FROM {symbol}', engine).values[0][0]
    print(max_date)
    new_data = yf.download(symbol, start = pd.to_datetime(max_date))
    new_rows = new_data[new_data.index > max_date]
    new_rows.to_sql(symbol, engine, if_exists='append')
    print(str(len())) + 'new rows imported to DB'